In [3]:
from PIL import Image
import pandas as pd
import jsonlines
import os



In [4]:
import torch

device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [4]:
from PIL import Image
import pandas as pd
import jsonlines
import os

parent_directory = os.path.dirname(os.path.dirname(os.getcwd()))
data_directory = os.path.join(parent_directory,'Hateful-Memes/data')

# Open the JSONL file in read mode using jsonlines
with jsonlines.open(os.path.join(data_directory,'dev.jsonl')) as reader:
    # Iterate over each line in the file
    for data in reader:
       
        print(data)

{'id': 8291, 'img': 'img/08291.png', 'label': 1, 'text': 'white people is this a shooting range'}
{'id': 46971, 'img': 'img/46971.png', 'label': 1, 'text': 'bravery at its finest'}
{'id': 3745, 'img': 'img/03745.png', 'label': 1, 'text': 'your order comes to $37.50 and your white privilege discount brings the total to $37.50'}
{'id': 83745, 'img': 'img/83745.png', 'label': 1, 'text': 'it is time.. to send these parasites back to the desert'}
{'id': 80243, 'img': 'img/80243.png', 'label': 1, 'text': 'mississippi wind chime'}
{'id': 5279, 'img': 'img/05279.png', 'label': 1, 'text': "knowing white people , that's probably the baby father"}
{'id': 1796, 'img': 'img/01796.png', 'label': 1, 'text': 'life hack #23 how to get stoned with no weed'}
{'id': 53046, 'img': 'img/53046.png', 'label': 1, 'text': "you've heard of elf on a shelf, now get ready for"}
{'id': 82301, 'img': 'img/82301.png', 'label': 1, 'text': 'cooooooooooooon!!!!'}
{'id': 31752, 'img': 'img/31752.png', 'label': 1, 'text': 

### Create Directories

### Load in Transformer model from HuggingFace

##### VisualBert and Tokenizer

In [5]:
import torch 
from transformers import BertTokenizer, VisualBertModel

model = VisualBertModel.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

##### Swinv2Model for image processing

In [1]:
from transformers import Swinv2Model,AutoImageProcessor

viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")#.from_pretrained('yusx-swapp/ofm-swinv2-base-patch4-window7-cifar100')
image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")#.from_pretrained('yusx-swapp/ofm-swinv2-base-patch4-window7-cifar100')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [2]:
processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")
model = AutoModelForImageClassification.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


NameError: name 'AutoModelForImageClassification' is not defined

In [3]:
from transformers import AutoImageProcessor, Swinv2Model
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
model = Swinv2Model.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

inputs = image_processor(image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for huggingface/cats-image contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/huggingface/cats-image
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


[1, 64, 768]

##### Create DataFrame from train.jsonl

In [7]:
import os 
import pandas as pd 
import jsonlines

parent_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(parent_directory,'data')

train_dataset = pd.DataFrame(columns=['id','img','label','text'])

# Open the JSONL file in read mode using jsonlines
with jsonlines.open(os.path.join(data_directory,'train.jsonl')) as reader:
    # Iterate over each line in the file
    for data in reader:
        # Process the data as needed
        data_df = pd.DataFrame([data])
        train_dataset = pd.concat([train_dataset,data_df]).reset_index(drop=True)


#### Swinv2 vs. ViT

In [8]:
from transformers import AutoImageProcessor, Swinv2Model
import torch
from datasets import load_dataset
import numpy as np
from PIL import Image
import os 

image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)


image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")
viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")

inputs = image_processor(tensor, return_tensors="pt")

with torch.no_grad():
    outputs = viz_model(**inputs)

visual_embeds = outputs.last_hidden_state
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
print(visual_embeds.shape)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


torch.Size([1, 36, 1024])


In [9]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
feature_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

inputs = feature_extractor(images=tensor, return_tensors="pt")
outputs = feature_model(**inputs)#.to('cuda'))

visual_embeds = outputs['last_hidden_state']
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
print(visual_embeds.shape)

/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


torch.Size([1, 197, 768])


#### Create input dataset for VisualBERT by connecting all the pieces

In [26]:
input_text = train_dataset.loc[0,'text']
input_text_tokenized = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
input_text_tokenized.update(
    {
        "visual_embeds": visual_embeds,
        "visual_token_type_ids": visual_token_type_ids,
        "visual_attention_mask": visual_attention_mask
    }
)


In [23]:
import torch 
from transformers import BertTokenizer, VisualBertModel

model = VisualBertModel.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
#model = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre')
output = model(**input_text_tokenized)
print(output)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (197x768 and 2048x768)

### Components of VisualBERT input

      1. Text input 
      2. Image input

    1. Text input requires three pieces: input_ids, attention_mask, token_type_ids

    - input_ids: represent the token IDs of the input tokens after tokenization. Each token in the input text is converted into a numerical ID based on the tokenizer's vocabulary.

    - attention_mask: indicates which tokens should be attended to during processing. Binary mask where each position corresponds to a token in the input.

    - token_type_ids: represent the segment IDs for token types in the context of sequence pairs.

    2. Image input requires three pieces: visual_embeds, visual_token_type_ids, visual_attention_mask

    - visual_embeds: represents the embeddings of visual features in the model.

    - visual_token_type_ids: represent the token type IDs for visual toekens. In multi-modal transformers distinguishes between tokens representing visual features and tokens representing text

    - vision_attention_mask: guide the attention mechanism for visual tokens. Specifies which tokens should be attended to during processing.

### Import Dataset Class

    Probably dont need what's below

In [2]:

import torch 
from transformers import BertTokenizer, VisualBertModel
from transformers import Swinv2Model,AutoImageProcessor
import os 
import pandas as pd
import jsonlines
from PIL import Image
import numpy as np


class DatasetBuilder():
    def __init__(self,model = None,tokenizer = None,json_file = None):
        self.parent_directory = os.path.dirname(os.getcwd())
        self.data_directory = os.path.join(self.parent_directory,'data')
        self.json_file = json_file
        print(tokenizer)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer)
        self.image_processor = AutoImageProcessor.from_pretrained(model)
        self.viz_model = Swinv2Model.from_pretrained(model)
        self.dataset = self.create_dataframe()

    def create_dataframe(self):
        dataset = pd.DataFrame(columns=['id','img','label','text'])
        with jsonlines.open(os.path.join(self.data_directory,self.json_file)) as reader:
            for data in reader:
                data_df = pd.DataFrame([data])
                dataset = pd.concat([dataset,data_df]).reset_index(drop=True)

        dataset['img'] = (self.data_directory+'/'+dataset['img']).apply(self.load_image)
        return dataset
    

    def tokenize_data(self,value):
        input = self.tokenizer(value['text'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
        target = torch.tensor(value['label']).type(torch.int64)
        
        
        image = self.image_processor(value['img'], return_tensors="pt")
            
            
        outputs = self.viz_model(**image)

        visual_embeds = outputs.last_hidden_state

       # except:
       #     visual_embeds = np.zeros(shape=(197, 768), dtype=float)

            
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
        input.update(
            {
                "visual_embeds": visual_embeds,
                "visual_token_type_ids": visual_token_type_ids,
                "visual_attention_mask": visual_attention_mask,
                "label":target
            }
        )

        return input
    
    def get_dataset(self):
        return self.dataset
  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset.loc[index])
        
        for k in inputs.keys():
            print(k, inputs[k].shape, inputs[k].dtype)

        return inputs

    def __len__(self):
        return len(self.dataset)

    def load_image(self,filepath):
        image = Image.open(filepath)
        image_array = np.array(image)
        return image_array

In [3]:
from transformers import TrainingArguments, Trainer
batch_size = 24
seq_len = 50

metric_name = "auroc"

args = TrainingArguments(
    output_dir = "model-checkpoint",
    seed = 110, 
    evaluation_strategy = "steps",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 100,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_steps = 50,
    save_steps = 500,
    fp16 = False,
    gradient_accumulation_steps = 2
)

/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:

from sklearn.metrics import roc_auc_score
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
print(metrics_list)


acc_metric = load_metric('accuracy')
f1_metric = load_metric('f1')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    auc_score = roc_auc_score(labels, predictions)
    return {"accuracy": acc['accuracy'], "auroc": auc_score,'f1':f1['f1'],'precision':precision['precision'],'recall':recall['recall']}

/tmp/ipykernel_23032/3211214775.py:3: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'character', 'charcut_mt', 'chrf', 'code_eval', 'comet', 'competition_math', 'confusion_matrix', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'mape', 'mase', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'nist_mt', 'pearsonr', 'perplexity', 'poseval', 'precision', 'r_squared', 'recall', 'rl_reliability', 'roc_auc', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'smape', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s', 'Aledade/extraction_evaluation', 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics', 'Bekhouche/NED', 'BucketHeadP65/confusion_matrix', 'BucketHeadP65/roc_curve', 'CZLC/rouge_raw', 'DaliaCaRo/accents_unplugged_eval', 'DarrenChensformer/action_generation', 'DarrenChensformer/eval_keyphrase', 'DarrenChensformer/relation_extraction', 'DoctorSlimm/bangalore_score', 'DoctorSlimm/ka

/tmp/ipykernel_23032/3211214775.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc_metric = load_metric('accuracy')
/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/datasets/l

In [5]:
from model.visbert import VisualBERT
import torch 
from transformers import BertTokenizer, VisualBertModel

model = VisualBERT()
tokenizer_name = 'google-bert/bert-base-uncased'
swin_model = "microsoft/swinv2-base-patch4-window12-192-22k"
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [6]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [7]:
trainer = Trainer(
    model,
    args,
    train_dataset = DatasetBuilder(DatasetBuilder(swin_model,tokenizer_name,'train.jsonl')),
    eval_dataset = DatasetBuilder(DatasetBuilder(swin_model,tokenizer_name,'test.jsonl')),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


google-bert/bert-base-uncased
None


OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [1]:
from data_setup.databuilder import DatasetBuilder

import torch 
from transformers import BertTokenizer, VisualBertModel
from transformers import Swinv2Model,AutoImageProcessor
import os 
import pandas as pd
import jsonlines
from PIL import Image
import numpy as np
from model.visbert import VisualBERT
from sklearn.metrics import roc_auc_score
from datasets import list_metrics, load_metric
from transformers import TrainingArguments, Trainer
batch_size = 24
seq_len = 50

metric_name = "auroc"

args = TrainingArguments(
    output_dir = "model-checkpoint",
    seed = 110, 
    evaluation_strategy = "steps",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 100,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_steps = 50,
    save_steps = 500,
    fp16 = False,
    gradient_accumulation_steps = 2
)




#metrics_list = list_metrics()
#print(metrics_list)


acc_metric = load_metric('accuracy')
f1_metric = load_metric('f1')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    auc_score = roc_auc_score(labels, predictions)
    return {"accuracy": acc['accuracy'], "auroc": auc_score,'f1':f1['f1'],'precision':precision['precision'],'recall':recall['recall']}




tokenizer = 'google-bert/bert-base-uncased'
model = "microsoft/swinv2-base-patch4-window12-192-22k"
#dataset = DatasetBuilder(model,tokenizer,'train.jsonl')



visual_model = VisualBERT()
swin_model = "microsoft/swinv2-base-patch4-window12-192-22k"
#visual_model.forward(dataset[0])

optimizer = torch.optim.Adam(visual_model.parameters(), )

trainer = Trainer(
    visual_model,
    args,
    train_dataset = DatasetBuilder(swin_model,tokenizer,json_file='train.jsonl'),
    eval_dataset = DatasetBuilder(swin_model,tokenizer,json_file='test.jsonl'),
    #tokenizer=BertTokenizer.from_pretrained("./tokenizer/", local_files_only=True),
    compute_metrics=compute_metrics
)


trainer.train()
#for i in range(1):
##    for j in dataset.get_dataset():
 #       optimizer.zero_grad()
  #      loss, _ = visual_model(j)
 #       loss.backward()
 #       optimizer.step()
 #       print(loss)


/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/tmp/ipykernel_45210/3443408764.py:44: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc_metric = load_metric('accuracy')
/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/da

google-bert/bert-base-uncased
/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/experiments


OSError: Incorrect path_or_model_id: './tokenizer/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [6]:
import os 
import pandas as pd 
import jsonlines

parent_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(parent_directory,'data')

train_dataset = pd.DataFrame(columns=['id','img','label','text'])

# Open the JSONL file in read mode using jsonlines
with jsonlines.open(os.path.join(data_directory,'train.jsonl')) as reader:
    # Iterate over each line in the file
    for data in reader:
        # Process the data as needed
        data_df = pd.DataFrame([data])
        train_dataset = pd.concat([train_dataset,data_df]).reset_index(drop=True)


In [1]:
from transformers import Swinv2Config, Swinv2Model

# Initializing a Swinv2 microsoft/swinv2-tiny-patch4-window8-256 style configuration
configuration = Swinv2Config()

# Initializing a model (with random weights) from the microsoft/swinv2-tiny-patch4-window8-256 style configuration
model = Swinv2Model(configuration)

# Accessing the model configuration
configuration = model.config

In [9]:
configuration

Swinv2Config {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    6,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 96,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swinv2",
  "num_channels": 3,
  "num_heads": [
    3,
    6,
    12,
    24
  ],
  "num_layers": 4,
  "out_features": [
    "stage4"
  ],
  "out_indices": [
    4
  ],
  "patch_size": 4,
  "pretrained_window_sizes": [
    0,
    0,
    0,
    0
  ],
  "qkv_bias": true,
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4"
  ],
  "transformers_version": "4.39.3",
  "use_absolute_embeddings": false,
  "window_size": 7
}

In [14]:
from transformers import AutoImageProcessor, Swinv2Model
import torch
from datasets import load_dataset
import numpy as np
from PIL import Image
import os 

image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)


image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")
viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-tiny-patch4-window16-256")

inputs = image_processor(tensor, return_tensors="pt")

with torch.no_grad():
    outputs = viz_model(**inputs)

visual_embeds = outputs.last_hidden_state
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
print(visual_embeds.shape)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


torch.Size([1, 36, 768])


In [11]:
from transformers import AutoImageProcessor, Swinv2Model
import torch
from datasets import load_dataset
import numpy as np
from PIL import Image
import os 

image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)


image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")
viz_model = Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")

inputs = image_processor(tensor, return_tensors="pt")

with torch.no_grad():
    outputs = viz_model(**inputs)

visual_embeds = outputs.last_hidden_state
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
print(visual_embeds.shape)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


torch.Size([1, 36, 1024])


In [20]:
from transformers import AutoImageProcessor, Swinv2ForImageClassification
from PIL import Image
import requests


image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)


image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
viz_model =Swinv2Model.from_pretrained("microsoft/swinv2-base-patch4-window12-192-22k")

inputs = image_processor(tensor, return_tensors="pt")

with torch.no_grad():
    outputs = viz_model(**inputs)

visual_embeds = outputs.last_hidden_state
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
print(visual_embeds.shape)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


torch.Size([1, 64, 1024])


KeyError: <class 'transformers.models.swinv2.configuration_swinv2.Swinv2Config'>

In [7]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
from transformers import AutoImageProcessor, Swinv2ForImageClassification
from PIL import Image
import requests
import os 
import pandas as pd 
import jsonlines
import numpy as np 

parent_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(parent_directory,'data')

train_dataset = pd.DataFrame(columns=['id','img','label','text'])

# Open the JSONL file in read mode using jsonlines
with jsonlines.open(os.path.join(data_directory,'train.jsonl')) as reader:
    # Iterate over each line in the file
    for data in reader:
        # Process the data as needed
        data_df = pd.DataFrame([data])
        train_dataset = pd.concat([train_dataset,data_df]).reset_index(drop=True)



image = Image.open(os.path.join(data_directory,train_dataset.loc[0,'img']))
image_array = np.array(image)
tensor = torch.tensor(image_array)



feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
feature_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

inputs = feature_extractor(images=tensor, return_tensors="pt")
outputs = feature_model(**inputs)#.to('cuda'))

visual_embeds = outputs['last_hidden_state']
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.int64)
print(visual_embeds.shape)

/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


torch.Size([1, 197, 768])


In [3]:
from transformers import BertTokenizer, VisualBertModel, TrainingArguments, Trainer, VisualBertConfig
configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre',
                                                hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.3)
model = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', config=configuration)

In [4]:
model = model.double()

In [5]:
model

VisualBertModel(
  (embeddings): VisualBertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (visual_token_type_embeddings): Embedding(2, 768)
    (visual_position_embeddings): Embedding(512, 768)
    (visual_projection): Linear(in_features=1024, out_features=768, bias=True)
  )
  (encoder): VisualBertEncoder(
    (layer): ModuleList(
      (0-11): 12 x VisualBertLayer(
        (attention): VisualBertAttention(
          (self): VisualBertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.3, inplace=False)
          )
          (output): V

In [2]:
import torch
torch.cuda.is_available()

/home/jramos/Documents/OMSCS/CS-7643 Deep Learning/Project/Hateful-Memes/workspace/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [3]:
torch.cuda.is_available()

False

In [4]:
torch.__version__

'2.2.2+cu121'